<a href="https://colab.research.google.com/github/charlie-paterson/West-Brom-Results-Machine-Learning/blob/main/West_brom_results.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from statsmodels.stats.outliers_influence import variance_inflation_factor
from sklearn.model_selection import train_test_split, GridSearchCV
import statsmodels.api as sm
import numpy as np
from sklearn import preprocessing
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report, ConfusionMatrixDisplay, confusion_matrix, accuracy_score
from sklearn.tree import DecisionTreeClassifier, plot_tree
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import roc_auc_score, f1_score
from xgboost import XGBClassifier
from sklearn.naive_bayes import GaussianNB, MultinomialNB, BernoulliNB
from imblearn.over_sampling import SMOTE
from sklearn.pipeline import Pipeline

## Loading Dataset

In [ ]:
df = pd.read_excel('West Brom.xlsx')
df

,Name,Number,POS,GP,GS,CS,SV,GA,A,PAcc,...,Unnamed: 18,Unnamed: 19,Unnamed: 20,Unnamed: 21,Unnamed: 22,Unnamed: 23,Unnamed: 24,Unnamed: 25,Unnamed: 26,Unnamed: 27
0,A. Palmer,24,GK,46,46,18,104,47,0,73.4,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,J.Griffiths,33,GK,0,0,0,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Name,Number,POS,GP,GS,G,A,SH,SOT,PAcc,...,Percent algorithm,Index,NaN,NaN,Results,Home,HG,AG,Away,Result
5,B. Thomas-Asante,21,A,39,31,11,2,52,31,70.2,...,0.874576,11,NaN,NaN,2023-08-05 00:00:00,Blackburn Rovers,2,1,West Bromwich Albion,L
6,J. Swift,19,M,37,20,9,1,39,15,85.8,...,0.593037,10,NaN,NaN,2023-08-12 00:00:00,West Bromwich Albion,3,2,Swansea City,W
7,A. Mowatt,27,M,43,35,2,5,17,6,89.9,...,0.687555,8,NaN,NaN,2023-08-18 00:00:00,Leeds United,1,1,West Bromwich Albion,D
8,J. Wallace,7,M,42,35,6,5,23,13,75.9,...,0.71929,9,NaN,NaN,2023-08-26 00:00:00,West Bromwich Albion,4,2,Middlesbrough,W
9,O. Yokuşlu,35,M,44,41,1,1,23,7,87.4,...,0.728401,7,NaN,NaN,2023-09-02 00:00:00,West Bromwich Albion,1,2,Huddersfield Town,L


## Clean Data

In [ ]:
def drop_columns_in_range(df, start_col, end_col):
    start_idx = df.columns.get_loc(start_col)
    end_idx = df.columns.get_loc(end_col)
    return df.drop(df.columns[start_idx:end_idx+1], axis=1)

df_dropped = drop_columns_in_range(df, 'Name', 'Unnamed: 21')
df_dropped2 = df_dropped.drop(df.index[[0,1,2,3,4]])
df_dropped2

,Unnamed: 22,Unnamed: 23,Unnamed: 24,Unnamed: 25,Unnamed: 26,Unnamed: 27
5,2023-08-05 00:00:00,Blackburn Rovers,2,1,West Bromwich Albion,L
6,2023-08-12 00:00:00,West Bromwich Albion,3,2,Swansea City,W
7,2023-08-18 00:00:00,Leeds United,1,1,West Bromwich Albion,D
8,2023-08-26 00:00:00,West Bromwich Albion,4,2,Middlesbrough,W
9,2023-09-02 00:00:00,West Bromwich Albion,1,2,Huddersfield Town,L
10,2023-09-16 00:00:00,Bristol City,0,0,West Bromwich Albion,D
11,2023-09-20 00:00:00,Watford,2,2,West Bromwich Albion,D
12,2023-09-23 00:00:00,West Bromwich Albion,0,0,Millwall,D
13,2023-09-30 00:00:00,Preston North End,0,4,West Bromwich Albion,W
14,2023-10-03 00:00:00,West Bromwich Albion,1,0,Sheffield Wednesday,W


In [ ]:
df_dropped2 = df_dropped2.rename(columns={'Unnamed: 22': 'Date', 'Unnamed: 23': 'Home',
                                          'Unnamed: 24': 'HG', 'Unnamed: 25': 'AG',
                                          'Unnamed: 26': 'Away', 'Unnamed: 27': 'Result'})
df_dropped2

,Date,Home,HG,AG,Away,Result
5,2023-08-05 00:00:00,Blackburn Rovers,2,1,West Bromwich Albion,L
6,2023-08-12 00:00:00,West Bromwich Albion,3,2,Swansea City,W
7,2023-08-18 00:00:00,Leeds United,1,1,West Bromwich Albion,D
8,2023-08-26 00:00:00,West Bromwich Albion,4,2,Middlesbrough,W
9,2023-09-02 00:00:00,West Bromwich Albion,1,2,Huddersfield Town,L
10,2023-09-16 00:00:00,Bristol City,0,0,West Bromwich Albion,D
11,2023-09-20 00:00:00,Watford,2,2,West Bromwich Albion,D
12,2023-09-23 00:00:00,West Bromwich Albion,0,0,Millwall,D
13,2023-09-30 00:00:00,Preston North End,0,4,West Bromwich Albion,W
14,2023-10-03 00:00:00,West Bromwich Albion,1,0,Sheffield Wednesday,W


In [ ]:
df_dropped3 = df_dropped2.drop(columns=['Date'])
df_dropped3.reset_index()

,index,Home,HG,AG,Away,Result
0,5,Blackburn Rovers,2,1,West Bromwich Albion,L
1,6,West Bromwich Albion,3,2,Swansea City,W
2,7,Leeds United,1,1,West Bromwich Albion,D
3,8,West Bromwich Albion,4,2,Middlesbrough,W
4,9,West Bromwich Albion,1,2,Huddersfield Town,L
5,10,Bristol City,0,0,West Bromwich Albion,D
6,11,Watford,2,2,West Bromwich Albion,D
7,12,West Bromwich Albion,0,0,Millwall,D
8,13,Preston North End,0,4,West Bromwich Albion,W
9,14,West Bromwich Albion,1,0,Sheffield Wednesday,W


## Feature Engineering

In [ ]:
df_dropped3['Home_or_Away'] = df_dropped3.apply(
    lambda row: 'Home' if row['Home'] == 'West Bromwich Albion' else 'Away' if row['Away'] == 'West Bromwich Albion' else 'Unknown',
    axis=1
)
df_dropped3

,Home,HG,AG,Away,Result,Home_or_Away
5,Blackburn Rovers,2,1,West Bromwich Albion,L,Away
6,West Bromwich Albion,3,2,Swansea City,W,Home
7,Leeds United,1,1,West Bromwich Albion,D,Away
8,West Bromwich Albion,4,2,Middlesbrough,W,Home
9,West Bromwich Albion,1,2,Huddersfield Town,L,Home
10,Bristol City,0,0,West Bromwich Albion,D,Away
11,Watford,2,2,West Bromwich Albion,D,Away
12,West Bromwich Albion,0,0,Millwall,D,Home
13,Preston North End,0,4,West Bromwich Albion,W,Away
14,West Bromwich Albion,1,0,Sheffield Wednesday,W,Home


## Encoding Data

In [ ]:
df_dropped3.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 46 entries, 5 to 50
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   Home          46 non-null     object
 1   HG            46 non-null     object
 2   AG            46 non-null     object
 3   Away          46 non-null     object
 4   Result        46 non-null     object
 5   Home_or_Away  46 non-null     object
dtypes: object(6)
memory usage: 2.3+ KB


In [ ]:
label = LabelEncoder()

In [ ]:
label.fit(df_dropped3.Home)
df_dropped3.Home = label.transform(df_dropped3.Home)

In [ ]:
label.fit(df_dropped3.Away)
df_dropped3.Away = label.transform(df_dropped3.Away)

In [ ]:
df_dropped3['HG'] = df_dropped3['HG'].astype(int)

In [ ]:
df_dropped3['AG'] = df_dropped3['AG'].astype(int)

In [ ]:
label.fit(df_dropped3.Result)
df_dropped3.Result = label.transform(df_dropped3.Result)

In [ ]:
label.fit(df_dropped3.Home_or_Away)
df_dropped3.Home_or_Away = label.transform(df_dropped3.Home_or_Away)

## Training

In [ ]:
features = ['Home', 'HG', 'AG', 'Away', 'Home_or_Away']
X = df_dropped3[features]
y = df_dropped3['Result']

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y,test_size=0.20,random_state = 365)

In [ ]:
X_train_array = np.asarray(X_train, dtype='float')
y_train_array = np.asarray(y_train, dtype='int')
X_test_array = np.asarray(X_test, dtype='float')
y_test_array = np.asarray(y_test, dtype='int')

### Logistic Regression Model

In [ ]:
mnlog_reg = sm.MNLogit(y_train_array, X_train_array)
mnlog_reg_results = mnlog_reg.fit()
print(mnlog_reg_results.summary())

Optimization terminated successfully.
         Current function value: 0.878871
         Iterations 6
                          MNLogit Regression Results                          
Dep. Variable:                      y   No. Observations:                   36
Model:                        MNLogit   Df Residuals:                       26
Method:                           MLE   Df Model:                            8
Date:                Wed, 31 Jul 2024   Pseudo R-squ.:                  0.1368
Time:                        17:26:54   Log-Likelihood:                -31.639
converged:                       True   LL-Null:                       -36.652
Covariance Type:            nonrobust   LLR p-value:                    0.2633
       y=1       coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
x1            -0.0802      0.098     -0.816      0.414      -0.273       0.112
x2             0.6836      0.

### Random Forest Model

In [ ]:
best_ccp_alpha = 0.01
rf = RandomForestClassifier()
rf.fit(X_train_array, y_train_array)
y_pred = rf.predict(X_test_array)

In [ ]:
cr = classification_report(y_test_array, y_pred)
print("\nClassification Report:")
print(cr)
print(" ")

f1 = f1_score(y_test_array, y_pred, average='weighted')
print(f"F1 Score: {np.round(f1, 2)}")

accuracy = accuracy_score(y_test_array, y_pred)
print(f"Accuracy Score: {np.round(accuracy, 2)}")

cv_scores = cross_val_score(rf, X, y, cv=5)
print("Mean CV score:", np.round(cv_scores.mean(), 2))


Classification Report:
              precision    recall  f1-score   support

           0       0.67      0.50      0.57         4
           1       1.00      0.75      0.86         4
           2       0.50      1.00      0.67         2

    accuracy                           0.70        10
   macro avg       0.72      0.75      0.70        10
weighted avg       0.77      0.70      0.70        10

 
F1 Score: 0.7
Accuracy Score: 0.7
Mean CV score: 0.63


## Gradient Boosting Classifier

In [ ]:
gb = GradientBoostingClassifier()
gb.fit(X_train_array, y_train_array)
y_pred_gb = gb.predict(X_test_array)

In [ ]:
cr = classification_report(y_test_array, y_pred_gb)
print("\nClassification Report:")
print(cr)
print(" ")

f1 = f1_score(y_test_array, y_pred_gb, average='weighted')
print(f"F1 Score: {np.round(f1, 2)}")

accuracy = accuracy_score(y_test_array, y_pred_gb)
print(f"Accuracy Score: {np.round(accuracy, 2)}")

cv_scores = cross_val_score(gb, X, y, cv=5)
print("Mean CV score:", np.round(cv_scores.mean(), 2))


Classification Report:
              precision    recall  f1-score   support

           0       0.80      1.00      0.89         4
           1       1.00      0.75      0.86         4
           2       1.00      1.00      1.00         2

    accuracy                           0.90        10
   macro avg       0.93      0.92      0.92        10
weighted avg       0.92      0.90      0.90        10

 
F1 Score: 0.9
Accuracy Score: 0.9
Mean CV score: 0.57


Prediction Model

In [ ]:
mapping = {1: 'L', 0: 'W', 2: 'D'}
vectorized_mapping = np.vectorize(mapping.get)
new_y_test_array = vectorized_mapping(y_test_array)
vectorized_mapping = np.vectorize(mapping.get)
new_y_pred_gb = vectorized_mapping(y_pred_gb)
comparison_df = pd.DataFrame({'Actual': new_y_test_array, 'Predicted': new_y_pred_gb})
print("\nComparison of Actual vs Predicted Labels:")
print(comparison_df)


Comparison of Actual vs Predicted Labels:
  Actual Predicted
0      L         L
1      W         W
2      D         D
3      L         L
4      W         W
5      L         L
6      D         D
7      W         W
8      L         W
9      W         W


In [ ]:
new_y_pred_gb
count = 0
for pred in new_y_pred_gb:
  if pred == 'L':
    count += 0
  elif pred == 'W':
    count += 3
  elif pred == 'D':
    count += 1

print(f"Total points from test size results: {count}")

Total points from test size results: 17
